## Pretrained GAN

In [1]:
import fastai
from fastai.vision import *
from fastai.callbacks import *
from fastai.vision.gan import *

Untar the data at `URLs.PETS` and save the path to `path`.

## Crappified data

In [3]:
from crappify import *

Set `il` to an `ImageList` from the folder `path_hr`. In parallel call `crappifier` with `path_lr` aand `path_hr` as arguments over `il.items`.

Set `bs,size` to `32, 128`. Set `arch` to a resnet34.

## Pre-train generator

Now let's pretrain the generator.

Set arch to a resnet34. Create an `ImageItemList` from the folder `path_lr`, split it by a random (10) percent with a random seed of 42. Assign it to `src`.

Create a function `get_data` that takes `src`, labels from a one-arg function that connects `path_hr` and `arg.name`, transform it with `get_transforms(max_zoom=2.)`, size is what you set above, and `tfm_y` should be `True` (Why is the last part important?); we should then call `databunch` using our `bs` from eaarlier, normalize with `imagenet_stats` and normalize the `y` values as well. Set `data.c` to 3 (can you explain why?) and return `data`.

Set `data_gen` to `get_data(bs, size)`.

Show 4 examples from the next batch.

In [10]:
%matplotlib inline

Set `wd` to `1e-3`.

Set the `y_range` to `(-3., 3.)`.

Set `loss_gen` to `MSELossFlat`.

Create a function `create_gen_learner` that creates a `unet_learner` that uses `data_gen`, `arch`, `wd`, sets `blur` to True (what does this do?), `norm_type=NormType.Weight` (what does this do?), `self_attention=True` (what does this do?), `y_range` and `loss_func=loss_gen`.

create_gen_learner and assign it to `learn_gen`.

Fit a cycle with 2 epochs and `pct_start=0.8`. What does `pct_start` do?

Unfreeze the model.

Train it again for another 3-epoch cycle with an lr slice from 1e-6 to 1e-3.

Show the results with 4 rows.

Save the learner to gen-pre2.

## Save generated images

Load the learner `gen-pre2`.

Set `name_gen` to `image_gen`, and set `path_gen` to `path/name_gen`.

Create `path_gen` (Hint: check out the `mkdir` method of a path object). What does `exist_ok` do?

Write a function `save_preds` that takes in a data loader `dl`, iterates over the batches in that data loader, predicts images over each batch, and saves each image to `path_gen` with the same name as it had in the original dataset. Take advantage of `dl.dataset.items` to get the list of names from the data loader. Hint: you'll have to do a bit of looping here.

Call `save_preds` on `data_gen.fix_dl`. What does `fix_dl` do?

Use `PIL` to open the first image in `path_gen`.

## Train critic

Set `learn_gen` to `None`. Call `gc.collect`.

Create a function `get_crit_data` that takes as arguments `classes`, `bs`, `size`. It will:
- Take an ImageList and split away a random 10% for validation with a random seed of 42
- Transform with the default transforms except `max_zoom=2.`, size=size
- turn it into a databunch with batch size bs anr normalize with imagenet stats
- set the number of categories to 3

set `data_crit` to the result of `get_crit_data` with arguments `[name_gen, 'images']`, `bs=bs`, `size=size`.  

Show a batch over 3 rows with ds type `Train` and imgsize=3.

Set `loss_critic` to `AdaptiveLoss` with argument `nn.BCEWithLogitsLoss()`. What is this doing?

Create a function `create_critic_learner` that takes arguments `data`, `metrics`. It should return a `Learner` object with arguments `data`, `gan_critic()`, `metrics=metrics`, `loss_function=loss_critic`, `wd=wd`. Can you explain what each of these is doing?

Use `create_critic_learner` to create a learner `learn_critic`.

Fit a 6-epoch cycle with a max lr of 1e-3.

Save the model as `critic-pre2`.

## GAN

Set `learn_crit` and `learn_gen` to `None`, and do another round of GPU garbage collection.

Load the crit data from dirs `['crappy', 'images']`, `bs=bs`, `size=size`.

Recreate the critic learner from the dad you just loaded with no metrics, and load `critic-pre2`.

Create a gen learner and load `gen-pre2`.

Create a `GANLearner`. There are a few pieces here:
- Create a partial function based on `AdaptiveGANSwitcher` where `critic_thresh` = 0.65. Can you explain what the critic threshold is?
- Create learner `learn` using `GANLearner.from_learners` with `learn_gen`, `learn_crit`, `weights_gen`=(1., 50.), `show_img=False`, `switcher=switcher`, `opt_func=partial(optim.Adam, betas=(0.,0.99))`, `wd=wd`. Explain what each of these means.
- Append a partial function based on `GANDiscriminativeLR` with `mult_lr=5.` to the learner's callback functions. What does this do? 

Set the `lr` to `1e-4`.

Fit for 40 epochs with the lr above. This will take about an hour.

Save the model as `gan-lc`.

Set learn.data to a dataset with batch size 16 and image size 192. Hint: use `get_data`.

Fit for 10 epochs at half the lr you last used.

Show the results over 16 rows.

Save the results to `gan-1c`.

## fin